# Automatic retrieval of page content

### Josep Monclús Carrasco

### University of Valencia

In [1]:
import getpass
import json
import requests
import operator
import selenium
import pandas as pd
from bs4 import BeautifulSoup
#from requestium import Session,Keys
import gender_guesser.detector as gg
#from gender import GenderDetector
import datetime
from elasticsearch import Elasticsearch
import time
from random import randint
from nameparser import HumanName
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [205]:
#Pages to scrap
pages = ["https://www.tusclasesparticulares.com"
        ,"https://buscatuprofesor.es"
        ,"https://www.superprof.es"
        ,"https://yoopies.es/profesor-particulares"
        ,"https://www.classgap.com"
        ,""]
#Consultas hechas de (nombre,html,elementos)

#Nombre,Foto,Valoracion,Precio/hora,#estudiantes,acerca_de_la_clase,sobre_persona,opiniones,edad
                 #Tusclasesparticulares
elementos_pag = [[f"/html/body/div/div/div/div/form/div/div/div/div/div/a" #Nom
                 ,f"/html/body/div/div/div/div/form/div/div/div/div/img" #Foto
                 ,"Nan" #Val
                 ,f"/html/body/div/div/div/div/form/div/div/div/div/div/div/div/p[2]/b" #Preu
                 ,"Nan" #n_estudiants
                 ,"Nan" #sobre la clase
                 ,f"/html/body/div/div/div/div/form/div/div/div/div/div/div" #sobre la persona
                 ,"/html/body/div/div/div/div/form/div/div/div/div/div/div/div/ul/li[1]/div[2]" #Opinions
                 ,"Nan"]#Edat
                 #Buscatuprofe
                ,[f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/h1" #Nom
                 ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/a/div/img" #Foto
                 ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/span[1]" #Val
                 ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/b" #Preu
                 ,"Nan" #n_estudiants
                 ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/p[3]" #sobre la clase
                 ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/p" #sobre la persona
                 ,[".review-text"] #Opinions
                 ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div[3]/div/div[3]/div/div[2]/div/p"] #Edat
                 #Superprof
                ,[f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/p" #Nom
                  ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/img" #Foto
                  ,f"/html/body/div/div/div/div/div/div/div/div/div/div/ul/li/span/span" #Valoració
                  ,f"/html/body/div/div/div/div/div/div/div/div/div/ul/li/span[2]" #Preu
                  ,f"/html/body/div/div/div/div/div/div/div/div/div/ul/li[3]/span[2]" #n_estudiants
                  ,f"/html/body/div/div/div/div/div/div/div/div[4]/p" #sobre la clase
                  ,f"/html/body/div/div/div/div/div/div/div/div[5]/p" #sobre la persona
                  ,[f"main-text"] #Opinions f"/html/body/div/div/div/div/div/div/div/div/ul/li/div/p"
                  ,"Nan"] #Edat
                 #Yoopie
                ,[f"/html/body/div/div/main/div/section/div/div/div/div/div[2]" #Nom
                 ,f"/html/body/div/div/main/div/section/div/div/div/div/div[1]" #Foto
                 ,f"/html/body/div/div/main/div/section/div/div/div/div/div[3]/span[2]" #Valoració
                 ,f"/html/body/div/div/main/div/section/div/aside/div/div/section/form/div/em[1]" #Preu
                 ,f"Nan" #n_estudiants
                 ,f"/html/body/div/div/main/div/section/div/div/div/section/p" #sobre la clase
                 ,f"Nan" #sobre la persona
                 ,f"Nan" #Opinions
                 ,f"Nan"] #Edat (està junt al nom)
                 #Classgap
                ,[f"/html/body/main/section/div/div/div/div/h1" #Nom
                 ,f"/html/body/main/section/div/div/div/aside/div/figure/div/div/img" #Foto
                 ,f"Nan" #Valoració
                 ,f"/html/body/main/section/div/div/div/aside/div/div/ul/li[2]/span[2]" #Preu
                 ,f"/html/body/main/section/div[2]/div/div[2]/div[2]/p" #n_estudiants
                 ,f"Nan" #sobre la clase
                 ,f"/html/body/main/section/div/section/div/p[2]" #sobre la persona
                 ,[f"m"] #Opinions
                 ,f"Nan"] #Edat (està junt al nom)
                ]
#Links de los perfiles de los candidatos
contenido_html = [f"html/body/div/div/div/div/div/div/div/div/table/tbody/tr/td/div/div/div/a"
                 ,f"/html/body/div/div/div/div/div/div/div/div/div/div/div/div/div/div/div/a"
                 ,f"/html/body/div/div/div/div/ul/li/a[1]"
                 ,f"/html/body/div/div/main/div/section/section/section/section/section/section/section/ul/li/a"
                 ,f"/html/body/main/section/div/section/div/article/div/hgroup/h3/a"]

#Consultas en tusclasesparticulares
consultas1 = [("entrenador_personal-valencia-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/valencia.aspx",contenido_html[0]),
             ("entrenador_personal-barcelona-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/barcelona.aspx",contenido_html[0]),
             ("entrenador_personal-madrid-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/madrid.aspx",contenido_html[0]),
             ("entrenador_personal-bilbao-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/bilbao.aspx",contenido_html[0]),
             ("entrenador_personal-granada-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/granada.aspx",contenido_html[0]),
             ("entrenador_personal-sevilla-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/sevilla.aspx",contenido_html[0]),
             ("entrenador_personal-zaragoza-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/zaragoza.aspx",contenido_html[0]),
             ("entrenador_personal-malaga-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/malaga.aspx",contenido_html[0]),
             ("entrenador_personal-murcia-tcp","https://www.tusclasesparticulares.com/profesores-entrenador-personal/murcia.aspx",contenido_html[0]),
             ("musica-valencia-tcp","https://www.tusclasesparticulares.com/profesores-musica/valencia.aspx",contenido_html[0]),
             ("musica-barcelona-tcp","https://www.tusclasesparticulares.com/profesores-musica/barcelona.aspx",contenido_html[0]),
             ("musica-madrid-tcp","https://www.tusclasesparticulares.com/profesores-musica/madrid.aspx",contenido_html[0]),
             ("musica-bilbao-tcp","https://www.tusclasesparticulares.com/profesores-musica/bilbao.aspx",contenido_html[0]),
             ("musica-granada-tcp","https://www.tusclasesparticulares.com/profesores-musica/granada.aspx",contenido_html[0]),
             ("musica-sevilla-tcp","https://www.tusclasesparticulares.com/profesores-musica/sevilla.aspx",contenido_html[0]),
             ("musica-zaragoza-tcp","https://www.tusclasesparticulares.com/profesores-musica/zaragoza.aspx",contenido_html[0]),
             ("musica-malaga-tcp","https://www.tusclasesparticulares.com/profesores-musica/malaga.aspx",contenido_html[0]),
             ("musica-murcia-tcp","https://www.tusclasesparticulares.com/profesores-musica/murcia.aspx",contenido_html[0])
            ]

#Consultas en buscatuprofesor
consultas_btp = [
    ("biologia-valencia-btp","https://buscatuprofesor.es/tutors/biologia/valencia/",contenido_html[1]),
    ("biologia-barcelona-btp","https://buscatuprofesor.es/tutors/biologia/barcelona/",contenido_html[1]),
    ("biologia-madrid-btp","https://buscatuprofesor.es/tutors/biologia/madrid/",contenido_html[1]),
    ("biologia-bilbao-btp","https://buscatuprofesor.es/tutors/biologia/bilbao/",contenido_html[1]),
    ("biologia-granada-btp","https://buscatuprofesor.es/tutors/biologia/granada/",contenido_html[1]),
    ("biologia-sevilla-btp","https://buscatuprofesor.es/tutors/biologia/sevilla/",contenido_html[1]),
    ("biologia-zaragoza-btp","https://buscatuprofesor.es/tutors/biologia/zaragoza/",contenido_html[1]),
    ("biologia-malaga-btp","https://buscatuprofesor.es/tutors/biologia/malaga/",contenido_html[1]),
    ("biologia-murcia-btp","https://buscatuprofesor.es/tutors/biologia/murcia/",contenido_html[1]),
    
    ("canto-valencia-btp","https://buscatuprofesor.es/tutors/canto/valencia/",contenido_html[1]),
    ("canto-barcelona-btp","https://buscatuprofesor.es/tutors/canto/barcelona/",contenido_html[1]),
    ("canto-madrid-btp","https://buscatuprofesor.es/tutors/canto/madrid/",contenido_html[1]),
    ("canto-bilbao-btp","https://buscatuprofesor.es/tutors/canto/bilbao/",contenido_html[1]),
    ("canto-granada-btp","https://buscatuprofesor.es/tutors/canto/granada/",contenido_html[1]),
    ("canto-sevilla-btp","https://buscatuprofesor.es/tutors/canto/sevilla/",contenido_html[1]),
    ("canto-zaragoza-btp","https://buscatuprofesor.es/tutors/canto/zaragoza/",contenido_html[1]),
    ("canto-malaga-btp","https://buscatuprofesor.es/tutors/canto/malaga/",contenido_html[1]),
    ("canto-murcia-btp","https://buscatuprofesor.es/tutors/canto/murcia/",contenido_html[1]),
    
    ("pintura-valencia-btp","https://buscatuprofesor.es/tutors/pintura/valencia/",contenido_html[1]),
    ("pintura-barcelona-btp","https://buscatuprofesor.es/tutors/pintura/barcelona/",contenido_html[1]),
    ("pintura-madrid-btp","https://buscatuprofesor.es/tutors/pintura/madrid/",contenido_html[1]),
    ("pintura-bilbao-btp","https://buscatuprofesor.es/tutors/pintura/bilbao/",contenido_html[1]),
    ("pintura-granada-btp","https://buscatuprofesor.es/tutors/pintura/granada/",contenido_html[1]),
    ("pintura-sevilla-btp","https://buscatuprofesor.es/tutors/pintura/sevilla/",contenido_html[1]),
    ("pintura-zaragoza-btp","https://buscatuprofesor.es/tutors/pintura/zaragoza/",contenido_html[1]),
    ("pintura-malaga-btp","https://buscatuprofesor.es/tutors/pintura/malaga/",contenido_html[1]),
    ("pintura-murcia-btp","https://buscatuprofesor.es/tutors/pintura/murcia/",contenido_html[1]),
    
    ("quimica-valencia-btp","https://buscatuprofesor.es/tutors/quimica-1/valencia/",contenido_html[1]),
    ("quimica-barcelona-btp","https://buscatuprofesor.es/tutors/quimica-1/barcelona/",contenido_html[1]),
    ("quimica-madrid-btp","https://buscatuprofesor.es/tutors/quimica-1/madrid/",contenido_html[1]),
    ("quimica-bilbao-btp","https://buscatuprofesor.es/tutors/quimica-1/bilbao/",contenido_html[1]),
    ("quimica-granada-btp","https://buscatuprofesor.es/tutors/quimica-1/granada/",contenido_html[1]),
    ("quimica-sevilla-btp","https://buscatuprofesor.es/tutors/quimica-1/sevilla/",contenido_html[1]),
    ("quimica-zaragoza-btp","https://buscatuprofesor.es/tutors/quimica-1/zaragoza/",contenido_html[1]),
    ("quimica-malaga-btp","https://buscatuprofesor.es/tutors/quimica-1/malaga/",contenido_html[1]),
    ("quimica-murcia-btp","https://buscatuprofesor.es/tutors/quimica-1/murcia/",contenido_html[1]),
    
    ("italiano-valencia-btp","https://buscatuprofesor.es/tutors/italiano/valencia/",contenido_html[1]),
    ("italiano-barcelona-btp","https://buscatuprofesor.es/tutors/italiano/barcelona/",contenido_html[1]),
    ("italiano-madrid-btp","https://buscatuprofesor.es/tutors/italiano/madrid/",contenido_html[1]),
    ("italiano-bilbao-btp","https://buscatuprofesor.es/tutors/italiano/bilbao/",contenido_html[1]),
    ("italiano-granada-btp","https://buscatuprofesor.es/tutors/italiano/granada/",contenido_html[1]),
    ("italiano-sevilla-btp","https://buscatuprofesor.es/tutors/italiano/sevilla/",contenido_html[1]),
    ("italiano-zaragoza-btp","https://buscatuprofesor.es/tutors/italiano/zaragoza/",contenido_html[1]),
    ("italiano-malaga-btp","https://buscatuprofesor.es/tutors/italiano/malaga/",contenido_html[1]),
    ("italiano-murcia-btp","https://buscatuprofesor.es/tutors/italiano/murcia/",contenido_html[1]),
]

#Consultas en superprof
consultas_sp = [("ingles-online","https://www.superprof.es/s/ingles,Espa%C3%B1a,,,1.html?s=4",contenido_html[2]),
             ("frances-online","https://www.superprof.es/s/frances,Espa%C3%B1a,,,1.html",contenido_html[2]),
             ("informatica-online","https://www.superprof.es/s/informatica,Espa%C3%B1a,,,1.html",contenido_html[2]),
             ("repaso_escolar-online","https://www.superprof.es/s/repaso-escolar,Espa%C3%B1a,,,1.html?s=4",contenido_html[2])]
#Consultas en classgap
consultas_cl = [("biologia-online1","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=1&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online2","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=2&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online3","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=3&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online4","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=4&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online5","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=5&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online6","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=6&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online7","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=7&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online8","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=8&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online9","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=9&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online10","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=10&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online11","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=11&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("biologia-online12","https://www.classgap.com/es/search?cg=0&c=11&kt=&Rows=10&Page=12&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online1","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=1&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online2","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=2&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online3","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=3&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online4","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=4&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online5","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=5&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online6","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=6&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online7","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=7&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online8","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=8&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online9","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=9&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online10","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=10&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online11","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=11&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("italiano-online12","https://www.classgap.com/es/search?cg=0&c=66&kt=&Rows=10&Page=12&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online1","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=1&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online2","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=2&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online3","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=3&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online4","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=4&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online5","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=5&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online6","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=6&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online7","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=7&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online8","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=8&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online9","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=9&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online10","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=10&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online11","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=11&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("canto-online12","https://www.classgap.com/es/search?cg=0&c=15&kt=&Rows=10&Page=12&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online1","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=1&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online2","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=2&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online3","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=3&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online4","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=4&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online5","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=5&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online6","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=6&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online7","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=7&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online8","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=8&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online9","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=9&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online10","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=10&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online11","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=11&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4]),
             ("programacion-online12","https://www.classgap.com/es/search?cg=0&c=95&kt=220&Rows=10&Page=12&pa=true&l=es&lvlc=&lvln=&dh=&tzo=-60&fm=0&children=false&dd=&dhn=&minp=0&mp=0&r=0&currid=1",contenido_html[4])]
#Consultas en yoopie
consultas = [
    ("matematicas-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=MATHEMATICS&c=valencia",contenido_html[3]),
    ("matematicas_beginer-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=MATHEMATICS&c=valencia&subjectLevel=BEGINNER",contenido_html[3]),
    ("matematicas_advanced-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=MATHEMATICS&c=valencia&subjectLevel=ADVANCED",contenido_html[3]),
    ("fisica-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=PHYSICS&c=valencia",contenido_html[3]),
    ("repaso_escolar_principiante-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=HOMEWORK&c=valencia&subjectLevel=BEGINNER",contenido_html[3]),
     ("guitarra-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=GUITAR&c=valencia&distance=30",contenido_html[3]),
    ("espanol-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=SPANISH&c=valencia&distance=30",contenido_html[3])
    
]


#Clases Privadas
#consultas = [
#    ("informatica-online","https://yoopies.es/busco-profesor-particulares/resultados?subject=COMPUTER&c=valencia",contenido_html[5]),
#    ("informatica-online","https://yoopies.es/busco-profesor-particulares/resultados?subject=HOMEWORK&c=valencia",contenido_html[5]),
#    ("ingles-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=ENGLISH&c=valencia",contenido_html[5]),
#    ("frances-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=FRENCH&c=valencia",contenido_html[5]),
#    ("repaso_escolar_principiante-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=HOMEWORK&c=valencia&subjectLevel=BEGINNER",contenido_html[5]),
#    ("repaso_escolar_intermedio-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=HOMEWORK&c=valencia&subjectLevel=INTERMEDIATE",contenido_html[5]),
#    ("repaso_escolar_avanzado-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=HOMEWORK&c=valencia&subjectLevel=ADVANCED",contenido_html[5]),
#    ("ingles_principiante-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=ENGLISH&c=valencia&subjectLevel=BEGINNER",contenido_html[5]),
#    ("ingles_intermedio-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=ENGLISH&c=valencia&subjectLevel=INTERMEDIATE",contenido_html[5]),
#    ("ingles_avanzado-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=ENGLISH&c=valencia&subjectLevel=ADVANCED",contenido_html[5]),
#    ("frances_principiante-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=FRENCH&c=valencia&subjectLevel=BEGINNER",contenido_html[5]),
#    ("frances_intermedio-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=FRENCH&c=valencia&subjectLevel=INTERMEDIATE",contenido_html[5]),
#    ("frances_avanzado-valencia-yp","https://yoopies.es/busco-profesor-particulares/resultados?subject=FRENCH&c=valencia&subjectLevel=ADVANCED",contenido_html[5])
#    ]

In [187]:
def login_page(url_page,headless=False):
    """
    Start the loging to the webpage
    """
    if headless:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Create session
        browser = webdriver.Chrome(executable_path="/Users/josepmoncluscarrasco/Documents/4rt-PRIMER-QUATRI/tfg/codigo/webdriver/chromedriver-3",options=chrome_options) 
    else:
        browser = webdriver.Chrome(executable_path="/Users/josepmoncluscarrasco/Documents/4rt-PRIMER-QUATRI/tfg/codigo/webdriver/chromedriver-3")
    # Get the page content
    browser.get(url_page)  
    return browser

In [207]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
def get_profesors_url(session, querys,superprof = False):
    """
    Returns a dictionary where each element has a list with the 30 profesors url
    Inputs:
        session: selenium.webdriver.chrome
        querys: List of each different query, 
                each element is a tuple with the namequery + query.html + the html elements
    Returns:
        prof_url: dict with 
    """
    urls = dict()
    for query in querys:
        print(query[0])
        time.sleep(randint(5,15))
        # List of top 30 professors candidates given a Subject
        session.get(query[1])
        session.execute_script("document.body.style.zoom='80%'")
        #Click the button
        time.sleep(2.1)
        ###
        ###
        time.sleep(5.5)
        if superprof:
            time.sleep(120)
            time.sleep(60)
        try:
            top_prof = session.find_elements(By.XPATH,query[2])
            prof_urls = [url.get_attribute('href') for url in top_prof]
            print(len(prof_urls))
        except:
            prof_urls = "Nan"
            print("No professors")
        urls[query[0]] = prof_urls
    return urls

In [ ]:
url_profesors = get_profesors_url(login_page(pages[1]),consultas_sp,superprof=True)

In [7]:
def gender_estimation(name):
    gender_gg = None
    gender_illinois = None
    gender_gd = None
    
    d = gg.Detector()
    # Predict gender by name
    gender_gg = d.get_gender(name)
    if "mostly" in gender_gg:
        gender_gg = gender_gg.split("_")
        gender_gg = gender_gg[1]
        

    try:
        response = requests.get('http://abel.lis.illinois.edu/cgi-bin/ethnea/search.py?Fname='+name+'&format=json')
        if response.status_code == 200:
            gender_illinois = json.loads(response.text.replace("'","\""))['Genni'].lower()
    
    finally:
        # Normalize values for the gender attribute  
        if gender_gd == 'f':
            gender_gd = 'female'
        elif gender_gd == 'm':
            gender_gd = 'male'
        else:
            gender_gd = 'unknown'
    
        if gender_illinois == 'f':
            gender_illinois = 'female'
        elif gender_illinois == 'm':
            gender_illinois = 'male'
        else:
            gender_illinois = 'unknown'
            
        # Collect gender estimations for the 3 different methods    
        # 'gd': gender_gd
        gender_estimations = {'gg': gender_gg, 'illinois': gender_illinois}
        
        # Count number of occurrencies
        sum_female = sum(value == 'female' for value in gender_estimations.values())
        sum_male = sum(value == 'male' for value in gender_estimations.values())
        sum_unknown = sum(value == 'unknown' for value in gender_estimations.values())
        
        # Collect gender occurrencies
        gender_occurrencies = {'female': sum_female, 'male': sum_male, 'unknown': sum_unknown}
        
        # Get the gender with higher number of occurrencies (by voting)
        gender_max_occurrencies = (max(gender_occurrencies.items(), key=operator.itemgetter(1))[0])
        
        return {'gender': gender_max_occurrencies}

In [8]:
import hashlib,scrubadub, scrubadub_spacy

def hasher(instr):
    digest = hashlib.sha256(str(instr).encode("utf-8")).hexdigest()
    return digest[10:50]

scrubber = scrubadub.Scrubber(post_processor_list=[
    scrubadub.post_processors.FilthReplacer(include_hash=True,
hash_salt='example', hash_length=3),
    scrubadub.post_processors.PrefixSuffixReplacer(prefix='{{', suffix='}}'),
])

scrubber.add_detector(scrubadub_spacy.detectors.SpacyNameDetector)

In [66]:
import json
from urllib.request import urlopen
myKey = "<your key>"
def gender(nom):
    url = "https://gender-api.com/get?key=" + myKey + "&name="+nom
    response = urlopen(url)
    decoded = response.read().decode('utf-8')
    data = json.loads(decoded)
    return data["gender"]

In [9]:
def nacionality_estimation(name):
    response = requests.get("https://api.nationalize.io/?name="+name+'&format=json')
    if response.status_code == 200:
        try:
            nacionality = json.loads(response.text.replace("'","\""))["country"][0]["country_id"]
        except:
            nacionality = "Nan"
    return nacionality

In [78]:
def get_profesor_information(session,prof_url,elements,yoopie = False,superprof = False,classgap = False):
    """
    Returns a list with the information regarding the professor
    """
    time.sleep(randint(1,3))
    prof_info = []
    time.sleep(1.7)
    try:
        session.get(prof_url)
        for i,element in enumerate(elements):
            time.sleep(0.5)
            #Then photo
            if i == 1:
                #identify image to be captured
                try:
                    l = session.find_element(By.XPATH,element)
                except:
                    l = "Nan"
                #Génere
                nombre = prof_info.pop().lstrip()
                prof_info.append(gender_estimation(nombre.split(" ")[0])["gender"])
                #Nacionalitat
                try:
                    prof_info.append(nacionality_estimation(nombre.split(" ")[0]))
                except:
                    prof_info.append("Nan")
                #prof_info.append(gender(nombre.split(" ")[0]))
                #Id hash
                prof_info.append(str(hasher(prof_url)))
                #Té foto
                if yoopie or classgap:
                    prof_info.append(1)
                else:
                    if l != "Nan":
                        has_photo = l.get_attribute('src')
                        prof_info.append(int(has_photo.split("/")[-1] not in ["no_images.png","avatarm.svg"]))
                    else:
                        prof_inf.append("Nan")
                #Edat
                if yoopie:
                    prof_info.append(nombre.split(" ")[2])

            else:
                try:
                    if not isinstance(element,list):
                        value = session.find_element(By.XPATH,element)
                        text = value.get_attribute('textContent')
                        if i != 0:
                            text = scrubber.clean(text)
                    else:
                        if not superprof:
                        #Some formats of comments are in different elements
                            text = ""
                            for e in element:
                                values = session.find_elements(By.CSS_SELECTOR,e)
                                for value in values:
                                    t = value.get_attribute('textContent')
                                    text += " " + t
                            text = scrubber.clean(text)
                        else:
                            text = ""
                            for e in element:
                                values = session.find_elements(By.CLASS_NAME,e)
                                for aux,value in enumerate(values):
                                    t = value.get_attribute('textContent')
                                    if (t != prof_info[8]) and (t != prof_info[7]):
                                        text += " " + t
                            text = scrubber.clean(text)

                except NoSuchElementException:
                    text = "Nan"
                prof_info.append(text)
    except:
        prof_info = ["NA","NA","NA","NA","NA","NA","NA","NA","NA","NA","NA"]
        print("bad")
    
    return prof_info

In [209]:
to_csv_lst = []
for urls in url_profesors.keys():
    for url in url_profesors[urls]:
        to_csv_lst.append([url,urls.split("-")[0],urls.split("-")[1]])

In [210]:
a = pd.DataFrame(to_csv_lst,columns=["url","materia","lugar"])
a = a[a.materia != "frances"]
a.to_csv("data/urls/superprof_extrem.csv")
a

,url,materia,lugar
0,https://www.superprof.es/mejora-ingles-con-cad...,ingles,online
1,https://www.superprof.es/nativo-anglosajon-ofr...,ingles,online
2,https://www.superprof.es/profesor-especializad...,ingles,online
3,https://www.superprof.es/profesora-ingles-con-...,ingles,online
4,https://www.superprof.es/profesor-ingles-nativ...,ingles,online
...,...,...,...
3940,https://www.superprof.es/estudiante-ade-clases...,repaso_escolar,online
3941,https://www.superprof.es/docente-fpe-ingles-co...,repaso_escolar,online
3942,https://www.superprof.es/soy-ingeniera-aliment...,repaso_escolar,online
3943,https://www.superprof.es/estudiante-universida...,repaso_escolar,online


In [155]:
a.groupby("materia").count()

,url,lugar
materia,,
biologia,349,349
canto,78,78
italiano,141,141
pintura,77,77
quimica,340,340


## Buscatuprofe

In [157]:
profesor_value = get_profesor_information(login_page(pages[1],True),a.url[1],elementos_pag[1])
profesor_value

/Users/josepmoncluscarrasco/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == '__main__':


['unknown',
 'ES',
 '8085c744915d31c3c1cded90cded30a3f56a3dcf',
 1,
 '5',
 '10 ',
 'Nan',
 'Nan',
 '¡Hola!Soy Edu, tengo 23 años y vivo en Valencia.Después de finalizar la carrera, cursé un {{ORGANIZATION-519}}, Celular y Genética para poder profundizar un poco más en esta maravillosa área. Sin embargo, siguiendo lo que realmente me motiva, decidí cursar el Máster Universitario en Profesor/a de Educación Secundaria para tener la posibilidad de enseñar aquello que una vez disfruté estudiando.Dispongo de cierta experiencia en el mundo de la enseñanza ya que, a lo largo de la carrera y máster, he participado en diversos proyectos de la Universidad que implicaban dar charlas de divulgación y clases a alumnos de instituto.Intento implementar, en la medida de lo posible, metodologías innovadoras y adaptadas al perfil del alumno, tratando de mantener un enfoque práctico y divertido. Tengo experiencia en el mundo de la Investigación en biología molecular, por lo que, con el fin de fomentar un 

In [ ]:
#profesors = []
for i,prof in enumerate(a.url[len(profesors):]):
    profesor_value = get_profesor_information(login_page(pages[1],True),prof,elementos_pag[1])
    profesors.append(profesor_value)
    print(i)
    if ((i+1) % 5) == 0:
        clear_output(wait=True)

In [181]:
len(profesors)
#profesors = profesors[0:776]

985

In [182]:
df = pd.DataFrame(profesors,columns=["genero","nacionalidad","id_profesor","has_photo","valoracion","precio","n_estudiantes","acerca_de","sobre_mi","comentarios","edad"])
df

,genero,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
0,female,TN,435f0695de897d367ceb83982b66960a8db82b6e,1,€/h,15,Nan,Soy biotecnóloga y me baso en mi forma de ente...,Estudié biotecnología y me especialicé en biot...,,Certificado carrera
1,unknown,ES,8085c744915d31c3c1cded90cded30a3f56a3dcf,1,5,10,Nan,Nan,"¡Hola!Soy Edu, tengo 23 años y vivo en Valenci...","Explica bien, me ayudaron las clases. La verd...",Título grado en Biología
2,female,CO,a32b6a2f81a520f123e3befae9fcb9264324ec89,1,€/h,15,Nan,Nan,"Cada persona tiene un aprendizaje diferente, p...",,bachillerato francésdoctorado {{ORGANIZATION-1...
3,female,KE,777d9845cf1033ac2391254867f3ba45cca2df7e,1,4.5,15,Nan,Nan,Graduada en biología con mención en biotecnolo...,La tutora es muy amable y puntual,18-25
4,male,CO,bf14c98665371e59c425cd3e34ecf07ee4ebc1b9,1,€/h,8,Nan,Preparo el tema con antelación y lo explico co...,"{{ORGANIZATION-251}}, graduado en {{ORGANIZATI...",,26-35
...,...,...,...,...,...,...,...,...,...,...,...
980,female,Nan,6be945418243e587964e919ff361a03a2b7b5c52,1,€/h,17,Nan,Nan,Estudié biología en {{ORGANIZATION-A70}}. Siem...,,26-35
981,female,Nan,be74da8e22f1301b8fd358237b631ab404d7f801,0,€/h,10,Nan,Nan,Soy pedagoga y técnico en educación infantil. ...,,26-35
982,female,Nan,0db54501cb6ce7300ff62905fda3e5f3b873f39a,0,€/h,10,Nan,Nan,He enseñado a muchos/as chicos/as el italian a...,,18-25
983,female,Nan,a0d70b4eead7246cbf6d3052efe1a25478ec1f83,1,€/h,10,Nan,Nan,"Soy italiana, licenciada en traducción e inter...",,18-25


In [183]:
df.groupby("genero").count()

,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
genero,,,,,,,,,,
NA,3,3,3,3,3,3,3,3,3,3
female,630,630,630,630,630,630,630,630,630,630
male,291,291,291,291,291,291,291,291,291,291
unknown,61,61,61,61,61,61,61,61,61,61


In [161]:
#df.groupby("nacionalidad").count()

In [184]:
df.precio[df.precio == "NA"] = "Nan"
df.precio = df.precio.astype(float)
df.groupby("genero").agg({'precio': ['mean']})

,precio
,mean
genero,
NA,NaN
female,14.498413
male,15.082474
unknown,14.377049


In [185]:
df.to_csv("data/buscatuprofesor-3-nuevo.csv")

## Superprof

In [14]:
a.iloc[0][0]

'https://www.superprof.es/clases-salsa-particular-profesora-cubana-titulada-aprende-soltar-cuerpo-descubrir-los-movimientos.html'

In [15]:
from IPython.display import clear_output 
clear_output(wait=True)

In [ ]:
from IPython.display import clear_output 
clear_output(wait=True)
profesor_value = get_profesor_information(login_page(pages[2],True),a.iloc[0][0],elementos_pag[2],superprof = True)
profesor_value

In [ ]:
profesors = []
for i,prof in enumerate(a.url[len(profesors):]):
    profesor_value = get_profesor_information(login_page(pages[2],True),prof,elementos_pag[2],superprof = True)
    profesors.append(profesor_value)
    print(i)
    if ((i+1) % 5) == 0:
        clear_output(wait=True)

In [260]:
import pandas as pd
import numpy as np
df = pd.DataFrame(profesors,columns=["genero","nacionalidad","id_profesor","has_photo","valoracion","precio","n_estudiantes","acerca_de","sobre_mi","comentarios","edad"])
df

,genero,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
0,female,Nan,b8440a13b0162bc8c6390bdb130ee5f22f69d799,1,5,40€,50+,"¡Hola, soy {{NAME-835}}! Nací en Eslovaquia, p...","¡Hola, soy {{NAME-835}}! Profesora particular ...",¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
1,male,Nan,97af4204e54a668d6a69e645887a20281d5927a2,1,5,18€,50+,Soy {{NAME-87F}} y un licenciado en {{ORGANIZA...,Doy clases de conversación en línea para estud...,¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
2,male,Nan,88227492cae6c095bc9a6e47d0809aee63a8af84,1,4.9,25€,50+,"Más de doce años, comencé a los 16 ayudando a ...","He estudiado literatura, lingüística y redacci...",¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
3,female,Nan,ec1d17c6ecbefa0b5a5c7654acf950ce311f0baf,1,4.9,13€,50+,Mi método de enseñanza es muy interactivo con ...,Nan,¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
4,male,Nan,f63e68ac2cc4ef72f972da3f92cfa39737c9bce4,1,5,20€,50+,Soy un Profesor de inglés certificado ({{ORGAN...,¿Estás aburrido de los ejercicios de relleno d...,¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
...,...,...,...,...,...,...,...,...,...,...,...
299,male,IT,df778c519fb7e44dcb41c17ee54c3eef550eec97,1,5,15€,Nan,Metodología flexible de acuerdo con las necesi...,Nan,¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
300,male,KE,da8d507408b91e8ea746ddf2d70f25a741ea1107,1,5,20€,5,¿Necesitas mejorar tu inglés para impresionar ...,Nan,¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
301,female,MX,bcf742c51f4e1eabdd35a8171c69cb13121b31c7,1,5,10€,2,Clases adaptadas para cada nivel y particula...,Nan,¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan
302,female,AR,a451661a5e611a0f15effd8a78922f1cb748ee86,1,5,9€,1,Serán clases de 60 minutos en las que por medi...,Nan,¿Qué quieres aprender? ¿Qué quieres aprender?...,Nan


In [250]:
df.groupby("genero").count()

,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
genero,,,,,,,,,,
NA,1,1,1,1,1,1,1,1,1,1
female,115,115,115,115,115,115,115,115,115,115
male,81,81,81,81,81,81,81,81,81,81
unknown,12,12,12,12,12,12,12,12,12,12


In [251]:
df.precio[df.precio == "NA"] = "Nan"
df.precio = df.precio.str.strip("€").astype(float)

In [230]:
df.groupby("genero").agg({'precio': ['mean']})

,precio
,mean
genero,
female,18.541401
male,19.075472
unknown,21.562500


In [252]:
df.to_csv("data/superprof4-extrem.csv")

## Yoopie

In [357]:
profesor_value = get_profesor_information(login_page(pages[3]),"https://yoopies.es/profesor-franc%C3%A9s/valencia/profesor-frances/4927049",elementos_pag[3],yoopie = True)

/Users/josepmoncluscarrasco/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # This is added back by InteractiveShellApp.init_path()


In [358]:
profesor_value

['unknown',
 '4d5998bccbb438',
 1,
 '31',
 '5.0',
 '20,00\xa0€',
 'Nan',
 'De origen francesa y española, soy diplomada en Doctorado en Estudios Hispánicos avanzados en {{ORGANIZATION-034}}. \nLlevo 10 años viviendo en Valencia.\n\nLlevo cinco años ejerciendo como profesor particular de francés en Valencia. \nAdemás, he realizado diversas formaciones en el ámbito docente y actualmente estoy trabajando en el liceo francés de Paterna.\n\nMe considero una persona afable, comprometida y trabajadora, con ganas de aportar a mis alumnos una gran motivación, una mejora de sus competencias y conociemientos.\nMe apasiona el aprendizaje de un idioma y naturalmente es un verdadero placer para mí impartir clases de frances. \n\nTambien imparto clases de apoyo para distintos niveles.\nA lo largo de mi experiencia, he perfeccionado mis habilidades de creación, innovación y autoaprendizaje.',
 'Nan',
 'Nan',
 'Nan']

In [31]:
profesors = []
for i,prof in enumerate(a["url"][0:]):
    profesor_value = get_profesor_information(login_page(pages[3],True),prof,elementos_pag[3],yoopie=True)
    profesors.append(profesor_value)
    print(i)
    if ((i+1) % 5) == 0:
        clear_output(wait=True)

80


In [32]:
profesors[0]

['female',
 'ES',
 '9089fc2db423882e469b67ccd03ac61e1a0eb3c9',
 1,
 '22',
 '5.0',
 '10,20\xa0€',
 'Nan',
 '{{NAME-F26}},\nSoy una universitaria con experiencia dando clases de refuerzo (siempre siguiendo la materia que están dando) y de ayuda con los deberes. Puedo dar clases de Lenguas (catalán y castellano) y {{NAME-1B9}}, y podría ayudar también con otras materias extras en caso de necesitarlo. Las clases serían para niños que cursen Primaria o hasta la ESO.\nAdemás, para poder ayudar mejor pediría (si se pudiera, sino no) poder revisar el día anterior lo que se trabajaría en la clase para así poder estar al día y que la clase fuera mejor (esto no lo cobraría, ya que sería algo por iniciativa propia para poder yo también explicarlo y ayudar de la mejor forma posible).',
 'Nan',
 'Nan',
 'Nan']

In [34]:
df = pd.DataFrame(profesors,columns=["genero","nacionalidad","id_profesor","has_photo","edad","valoracion","precio","n_estudiantes","acerca_de","sobre_mi","comentarios","trash"])
df

,genero,nacionalidad,id_profesor,has_photo,edad,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,trash
0,female,ES,9089fc2db423882e469b67ccd03ac61e1a0eb3c9,1,22,5.0,"10,20 €",Nan,"{{NAME-F26}},\nSoy una universitaria con exper...",Nan,Nan,Nan
1,female,IT,c9198242af1442db2f8e2f07bae1995be90dd3bd,1,29,5.0,"18,00 €",Nan,"{{NAME-F26}}, \nMi nombre es {{NAME-8F3}}, soy...",Nan,Nan,Nan
2,female,ES,991ed3f0f8f0ac55529b5c5ac3597ce1ab6922c1,1,52,5.0,"12,20 €",Nan,Hola soy {{NAME-9FC}} y doy clases de PRIMARIA...,Nan,Nan,Nan
3,female,PT,cc93c20783a141562a8f8b043e88a19569919ccf,1,22,5.0,"18,00 €",Nan,"Hola, soy {{NAME-E64}}, he estudiado la licenc...",Nan,Nan,Nan
4,male,SE,d6a9f6c97857c0364c04f8b0ab7fe91b7bcff6fc,1,30,5.0,"19,20 €",Nan,"Hello,\nI have been a mathematics tutor since ...",Nan,Nan,Nan
...,...,...,...,...,...,...,...,...,...,...,...,...
76,female,AR,0e8c19153b06b342f34e9fba3992785980057af2,1,40,5.0,"10,20 €",Nan,"Hola,\n\nSoy española nativa.\n\nImparto clase...",Nan,Nan,Nan
77,female,AR,9ccd7f92eaa8b223b6cab14ddf5c1c49dd9dc0f9,1,30,5.0,"20,20 €",Nan,"¡Hola!\nMi nombre es {{NAME-668}}, tengo 30 añ...",Nan,Nan,Nan
78,female,CR,d9276c43f24ff0a5093ae7257137aac400df661c,1,26,5.0,"12,20 €",Nan,"¡Hola! Mi nombre es {{NAME-9DC}}, tengo 26 año...",Nan,Nan,Nan
79,male,GB,48e76177ba585595d23fe1c95c4f974649d2747d,1,47,5.0,"14,00 €",Nan,"{{NAME-F26}}, \n\nSoy profesor de español desd...",Nan,Nan,Nan


In [35]:
#df.to_csv("data/yoopie2.csv")

## Tus clases particulares

In [ ]:
profesors = []
for i,prof in enumerate(a["url"][0:]):
    profesor_value = get_profesor_information(login_page(pages[0],True),prof,elementos_pag[0])
    profesors.append(profesor_value)
    print(i)
    if ((i+1) % 5) == 0:
        clear_output(wait=True)

In [104]:
len(profesors)

360

In [105]:
df = pd.DataFrame(profesors,columns=["genero","nacionalidad","id_profesor","has_photo","valoracion","precio","n_estudiantes","acerca_de","sobre_mi","comentarios","edad"])
df

,genero,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
0,female,TH,92384d46b76fcedc404ad7c1d5ebae1e0fef54bf,1,Nan,12,Nan,Nan,\n \n ...,Nan,Nan
1,female,TH,ab6938d1fb2f71700e0c048d64ce5ea355703679,1,Nan,20,Nan,Nan,\n \n ...,Nan,Nan
2,male,CO,8a1bde13a21055823209b0364d83cb7b1e9b2784,1,Nan,25,Nan,Nan,\n \n ...,Nan,Nan
3,female,PT,369360c38dcb873802465584fde6e0edc4a8cca0,1,Nan,10,Nan,Nan,\n \n ...,Nan,Nan
4,female,TH,faf957de4e6b6d1548fe78162b48f39e30ce7063,1,Nan,6,Nan,Nan,\n \n ...,Nan,Nan
...,...,...,...,...,...,...,...,...,...,...,...
355,male,PE,9dfa36eb83d7ef8e59cfa1a7cb2bd80007fc7634,1,Nan,20,Nan,Nan,\n \n ...,Nan,Nan
356,female,PT,5fe0ad2b4cd5533d8633080c8d4b3ce87bf0e6c7,1,Nan,18,Nan,Nan,\n \n ...,Nan,Nan
357,female,ES,f8254a48d3c843ce8d9031380af4e8d05293607b,1,Nan,30,Nan,Nan,\n \n ...,Nan,Nan
358,female,CR,22e3986bcb4f978bde3df210625c2bfbb0dcb113,1,Nan,12,Nan,Nan,\n \n ...,Nan,Nan


In [106]:
df.groupby("genero").count()

,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
genero,,,,,,,,,,
female,145,145,145,145,145,145,145,145,145,145
male,138,138,138,138,138,138,138,138,138,138
unknown,77,77,77,77,77,77,77,77,77,77


In [107]:
df.precio = df.precio.astype(float)

In [108]:
df.groupby("genero").agg({'precio': ['mean']})

,precio
,mean
genero,
female,20.472000
male,18.785185
unknown,17.833333


In [109]:
df.to_csv("data/tusclasesparticulares-nuevo.csv")

## Classgap

In [192]:
info = get_profesor_information(login_page(pages[4],True),a["url"][2],elementos_pag[4],superprof = True,classgap=True)
print(info)
print(a["url"][2])

/Users/josepmoncluscarrasco/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == '__main__':
/Users/josepmoncluscarrasco/opt/anaconda3/lib/python3.7/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


['male', 'Nan', '47a4e0b3320458f93598e78bebf4b771431d648c', 1, 'Nan', '$ 21/clase', '5 Valoraciones', 'Nan', 'Nan', '   Gran profesor! Clases muy bien preparadas en funcion del nivel del alumno y dadas de forma muy amena haciendo que sus clases sean muy faciles de seguir! Tambien es muy paciente y si te cuesta la materia te lo explica de todas las maneras posibles hasta que lo sacas. Muy recomendable!', 'Nan']
https://www.classgap.com/es/tutor/nacho-sanchez-rodes


In [197]:
#profesors = []
for i,prof in enumerate(a["url"][len(profesors):]):
    profesor_value = get_profesor_information(login_page(pages[4],True),prof,elementos_pag[4],superprof = True,classgap=True)
    profesors.append(profesor_value)
    print(i)
    if ((i+1) % 5) == 0:
        clear_output(wait=True)

270
271
272
273
274


In [198]:
len(profesors)

391

In [199]:
df = pd.DataFrame(profesors,columns=["genero","nacionalidad","id_profesor","has_photo","valoracion","precio","n_estudiantes","acerca_de","sobre_mi","comentarios","edad"])
df

,genero,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
0,male,Nan,862b320641912f85fd1a7468ab1f374a3f289b2d,1,Nan,$ 14/clase,291 Valoraciones,Nan,Nan,"Muy buena profesora , gracias a ella pude ava...",Nan
1,female,Nan,7cfd6caa63eabff99953577e79f2e5ff61803fe2,1,Nan,$ 16,2 Valoraciones,Nan,Nan,Me ha explicado muy bien,Nan
2,male,Nan,47a4e0b3320458f93598e78bebf4b771431d648c,1,Nan,$ 21/clase,5 Valoraciones,Nan,Nan,Gran profesor! Clases muy bien preparadas e...,Nan
3,male,Nan,f3c05baf6808bb8f62d05e012e39f4bf84bf8065,1,Nan,$ 19,33 Valoraciones,Nan,Nan,excellent first lesson! una leccion primera e...,Nan
4,female,Nan,c64c1f1b2312db6aeed479de0b264e13f423a7b4,1,Nan,$ 20,91 Valoraciones,Nan,Nan,"Patri es una profe atenta, divertida y explic...",Nan
...,...,...,...,...,...,...,...,...,...,...,...
386,male,PE,e875ece71914f542e0b348718a1c3b7573abdde7,1,Nan,$ 9/clase,11 Valoraciones,Nan,Estudiante de Ing. Química con fuertes conoc...,"Exelente, totalmente recomendado, muy buen in...",Nan
387,male,PE,fa7cd336ad42a4bb5f1e9249e45f7a6db9a1f6d1,1,Nan,$ 11/clase,3 Valoraciones,Nan,Tengo 8 años de experiencia dando clases onlin...,Buen profesor con conocimientos avanzados.,Nan
388,female,CR,8b2503b0e39d450baed5db31cf6cb7f8ac6587cf,1,Nan,$ 15/clase,Nan,Nan,"Hola, buenas tardes, días, o noches, mi nombre...",,Nan
389,male,CO,d6756a97685e27d3a31defded7b73cbee866ec68,1,Nan,$ 5,Nan,Nan,Nan,,Nan


In [200]:
df.groupby("genero").count()

,nacionalidad,id_profesor,has_photo,valoracion,precio,n_estudiantes,acerca_de,sobre_mi,comentarios,edad
genero,,,,,,,,,,
NA,1,1,1,1,1,1,1,1,1,1
female,253,253,253,253,253,253,253,253,253,253
male,120,120,120,120,120,120,120,120,120,120
unknown,17,17,17,17,17,17,17,17,17,17


In [ ]:
df.precio = df.precio.str.strip("$|€").str.replace(r'[a-z]|/',"").str.replace(r'N',"Nan").replace("NanA","Nan").astype(float)

In [203]:
df.groupby("genero").agg({'precio': ['mean']})

,precio
,mean
genero,
NA,NaN
female,17.687747
male,18.091667
unknown,16.764706


In [204]:
df.to_csv("data/classgap3.csv") 